In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [6]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=test_transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                          shuffle=True, num_workers=2)

Files already downloaded and verified


In [7]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [19]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self, num_classes=10):
          super(Net, self).__init__()
          self.layer1 = nn.Sequential(
              nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
              nn.BatchNorm2d(16),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=2))
          self.layer2 = nn.Sequential(
              nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
              nn.BatchNorm2d(32),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=2))
          self.fc1 = nn.Linear(8*8*32, 64)
          self.fc2 = nn.Linear(64, 128) #num_classes
          self.fc3 = nn.Linear(128, num_classes)
          self.softmax = nn.LogSoftmax()
          self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
          out = self.dropout(self.layer1(x))
          out = self.dropout(self.layer2(out))
          out = out.reshape(out.size(0), -1)
          out = self.dropout(self.fc1(out))
          out = self.dropout(self.fc2(out))
          out = self.fc3(out)
          return F.log_softmax(out, dim=1)

net = Net().to(device)

In [21]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [22]:
from tqdm import tqdm

epochs = 5
total_step = len(trainloader)

for epoch in range(epochs):
  running_loss = 0.0

  for i,data in enumerate(trainloader):
    inputs,labels = data[0].to(device), data[1].to(device)

    output = net(inputs)
    loss = criterion(output,labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    running_loss += loss.item()
    if (i+1) % 2000 == 1999:
      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [1999/12500], Loss: 1.7752
Epoch [1/5], Step [3999/12500], Loss: 2.0318
Epoch [1/5], Step [5999/12500], Loss: 1.1742
Epoch [1/5], Step [7999/12500], Loss: 3.3845
Epoch [1/5], Step [9999/12500], Loss: 1.7303
Epoch [1/5], Step [11999/12500], Loss: 1.3686
Epoch [2/5], Step [1999/12500], Loss: 0.3384
Epoch [2/5], Step [3999/12500], Loss: 0.5904
Epoch [2/5], Step [5999/12500], Loss: 1.2700
Epoch [2/5], Step [7999/12500], Loss: 0.7576
Epoch [2/5], Step [9999/12500], Loss: 0.6605
Epoch [2/5], Step [11999/12500], Loss: 0.5418
Epoch [3/5], Step [1999/12500], Loss: 1.3913
Epoch [3/5], Step [3999/12500], Loss: 0.7527
Epoch [3/5], Step [5999/12500], Loss: 0.7835
Epoch [3/5], Step [7999/12500], Loss: 0.8849
Epoch [3/5], Step [9999/12500], Loss: 1.0801
Epoch [3/5], Step [11999/12500], Loss: 0.6394
Epoch [4/5], Step [1999/12500], Loss: 1.6524
Epoch [4/5], Step [3999/12500], Loss: 0.5143
Epoch [4/5], Step [5999/12500], Loss: 0.9938
Epoch [4/5], Step [7999/12500], Loss: 0.4111
Epoch [

In [23]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

# **Prediction time**

In [24]:
net = Net().to(device)
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [25]:
correct=0
total=0

net.eval()
with torch.no_grad():
  for data in testloader:
    images,labels = data[0].to(device), data[1].to(device)
    
    outputs = net(images)
    _ , predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted==labels).sum().item()

  print('Accuracy of the network on the 10000 test images: {}'.format(correct*100/total))


Accuracy of the network on the 10000 test images: 71.26
